In [1]:
import numpy as np
import pandas as pd
import requests
import datetime
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

np.random.seed(123)

2024-09-25 22:58:55.470610: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-25 22:58:55.559293: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-25 22:58:55.623704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 22:58:56.450244: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 22:58:56.572479: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 22:59:03.321808: W tensorflow/compiler/tf2tensorrt/utils/p

# Categorical Encoding

In [14]:
# Load your generated data
df = pd.read_csv("Yield_shelile.csv")

# Encode categorical features (village and crop_type)
village_encoder = LabelEncoder()
df['village_encoded'] = village_encoder.fit_transform(df['village'])

crop_encoder = LabelEncoder()
df['crop_type_encoded'] = crop_encoder.fit_transform(df['crop_type'])
df.head(2)

,Unnamed: 0,village,timestamp,crop_type,total_yield,pesticides_tonnes,rainfall_mm,temperature_C,humidity_%,wind_speed_kmph,...,local_market_demand_tonnes,export_market_demand_tonnes,local_market_price_Maloti_per_tonne,export_market_price_Maloti_per_tonne,storage_cost_Maloti_per_month,transport_cost_Maloti_per_km,profit_margin_%,supply_vs_demand_ratio,village_encoded,crop_type_encoded
0,0,Thaba-Tseka,2020-01-31,wheat,8.08987,4.893046,145.975084,16.929255,65.696400,12.924528,...,2194.938184,1307.779627,2345.457957,6922.969040,238.905260,37.705094,5.002796,1.274152,4,2
1,1,Semonkong,2020-02-29,wheat,7.92908,4.649022,100.097834,31.787334,64.309958,8.381335,...,658.217938,969.927378,2610.217763,6012.231565,428.743783,20.621525,16.424340,0.682203,3,2


# Drop unwanted Features

In [24]:
unwanted = [
    "village",
    "crop_type",
    "soil_ph",
    "soil_moisture_%",
    "soil_nitrogen_ppm",
    "soil_phosphorus_ppm",
    "soil_potassium_ppm",
    "organic_matter_%",
    
    "local_market_demand_tonnes",
    "export_market_demand_tonnes",
    "local_market_price_Maloti_per_tonne",
    "export_market_price_Maloti_per_tonne",
    "storage_cost_Maloti_per_month",
    "transport_cost_Maloti_per_km",
    "profit_margin_%",
    "supply_vs_demand_ratio",

    # "fertilizer_kg_per_hectare",
    "irrigation_mm",
    "pesticide_use_liters",
    "NDVI",
    "EVI",
    "SAVI",
    "canopy_temp_C",
    "previous_yield_tonnes",
    "yield_variability_%",
    "market_price_usd_per_tonne",
    "demand_forecast_tonnes",
    "altitude_m",
    "slope_%",
    "proximity_to_water_km",
    # "drought_frequency",
    "heatwave_days",
    "CO2_concentration_ppm",
    # "pesticides_tonnes"
    ]

df.drop(columns=unwanted)

,Unnamed: 0,timestamp,total_yield,pesticides_tonnes,rainfall_mm,temperature_C,humidity_%,wind_speed_kmph,solar_radiation_Wm2,frost_days,fertilizer_kg_per_hectare,drought_frequency,village_encoded,crop_type_encoded
0,0,2020-01-31,8.089870,4.893046,145.975084,16.929255,65.696400,12.924528,612.751247,0,139.176232,2,4,2
1,1,2020-02-29,7.929080,4.649022,100.097834,31.787334,64.309958,8.381335,352.017584,3,52.374051,0,3,2
2,2,2020-03-31,5.815598,4.491830,103.043236,21.281160,82.610250,18.815323,613.643644,3,125.614008,2,0,2
3,3,2020-04-30,3.884584,4.150748,50.864971,22.464917,80.897286,12.460748,355.249260,1,100.383660,0,2,2
4,4,2020-05-31,3.876388,1.466555,74.863659,31.887018,63.528934,15.458833,570.209939,2,106.669691,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,163,2024-04-30,4.985180,2.690964,162.175922,33.317820,69.038793,13.788290,674.576948,4,52.628564,0,2,0
164,164,2024-05-31,5.864167,1.779257,100.447103,16.366858,68.957194,11.500750,400.784208,0,79.222839,1,1,0
165,165,2024-06-30,4.676492,3.836489,159.267031,22.732898,57.840831,17.222245,467.004067,4,111.210846,0,3,0
166,166,2024-07-31,8.911626,2.479495,161.250655,25.486653,77.891268,17.217433,462.248836,4,79.765129,1,2,0
